# 05 — Kalibrasyon (Calibration)

Kalibrasyon, modelin **olasılık çıktılarının gerçek frekansı ne kadar yansıttığını** ölçer.
- P(iptal)=0.8 diyen kayıtların %80'i gerçekten iptal olmalı
- Kötü kalibre model: maliyet hesabı bozulur, iş kararları güvenilmez olur

Bu notebook:
- Isotonic ve Sigmoid kalibrasyon karşılaştırması
- Reliability diagram (kalibre eğrisi)
- ECE (Expected Calibration Error) hesabı
- Kalibre edilmiş modeli kaydetme

In [ ]:
import sys

sys.path.insert(0, "..")

import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import CalibrationDisplay

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False

## 1. Model & Veri Yükleme

In [ ]:
from src.config import Paths
from src.io import load_latest_model, read_input_dataset
from src.preprocess import preprocess_basic
from src.split import split_dataset
from sklearn.model_selection import train_test_split

paths = Paths()
model = load_latest_model(paths)
print(f"Temel model: {type(model).__name__}")

df, _ = read_input_dataset(paths.raw_data)
df = preprocess_basic(
    df, target_col="is_canceled", label_map={"no": 0, "yes": 1, 0: 0, 1: 1}
)
split = split_dataset(df, target_col="is_canceled")
X_train, y_train = split.X_train, split.y_train
X_test, y_test = split.X_test, split.y_test

# Kalibrasyon için train'in bir bölümünü ayır (leakage önlemi)
X_cal, X_eval, y_cal, y_eval = train_test_split(
    X_test, y_test, test_size=0.5, stratify=y_test, random_state=42
)
print(f"Cal: {X_cal.shape}, Eval: {X_eval.shape}")

## 2. Kalibrasyon Öncesi Reliability Diagram

In [ ]:
y_prob_raw = model.predict_proba(X_test)[:, 1]

fig, ax = plt.subplots()
CalibrationDisplay.from_predictions(
    y_test, y_prob_raw, n_bins=10, ax=ax, name="Kalibre Edilmemiş", color="#ef4444"
)
ax.set_title("Reliability Diagram — Kalibre Edilmemiş Model")
plt.tight_layout()
plt.show()


# ECE hesabı
def compute_ece(y_true, y_prob, n_bins=10):
    bins = np.linspace(0, 1, n_bins + 1)
    ece = 0.0
    for i in range(n_bins):
        mask = (y_prob >= bins[i]) & (y_prob < bins[i + 1])
        if mask.sum() > 0:
            ece += (mask.sum() / len(y_true)) * abs(
                y_prob[mask].mean() - y_true[mask].mean()
            )
    return ece


ece_raw = compute_ece(y_test.values, y_prob_raw)
print(f"ECE (ham): {ece_raw:.4f}")

## 3. Isotonic vs Sigmoid Kalibrasyon

In [ ]:
from src.calibration import calibrate_frozen_classifier

cal_isotonic = calibrate_frozen_classifier(model, X_cal, y_cal, method="isotonic")
cal_sigmoid = calibrate_frozen_classifier(model, X_cal, y_cal, method="sigmoid")

y_prob_iso = cal_isotonic.calibrated_model.predict_proba(X_eval)[:, 1]
y_prob_sig = cal_sigmoid.calibrated_model.predict_proba(X_eval)[:, 1]

ece_iso = compute_ece(y_eval.values, y_prob_iso)
ece_sig = compute_ece(y_eval.values, y_prob_sig)

print(f"ECE Isotonic:  {ece_iso:.4f}")
print(f"ECE Sigmoid:   {ece_sig:.4f}")
print(f"ECE Ham model: {ece_raw:.4f} (referans)")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

CalibrationDisplay.from_predictions(
    y_eval, y_prob_raw, n_bins=10, ax=axes[0], name="Ham", color="#ef4444"
)
CalibrationDisplay.from_predictions(
    y_eval, y_prob_iso, n_bins=10, ax=axes[1], name="Isotonic", color="#10b981"
)
CalibrationDisplay.from_predictions(
    y_eval, y_prob_sig, n_bins=10, ax=axes[2], name="Sigmoid", color="#3b82f6"
)

for ax, title, ece in zip(
    axes, ["Ham", "Isotonic", "Sigmoid"], [ece_raw, ece_iso, ece_sig]
):
    ax.set_title(f"{title} (ECE={ece:.4f})")

plt.suptitle("Reliability Diagrams — Kalibrasyon Karşılaştırması", y=1.02)
plt.tight_layout()
plt.show()

## 4. En İyi Kalibrasyonu Kaydet

In [ ]:
import joblib

best_method = "isotonic" if ece_iso <= ece_sig else "sigmoid"
best_cal = cal_isotonic if best_method == "isotonic" else cal_sigmoid
print(f"En iyi kalibrasyon: {best_method}")

# Resmi kayıt için:
# python main.py calibrate
#
# Notebook'tan:
joblib.dump(
    best_cal.calibrated_model, f"../models/baseline_calibrated_{best_method}.joblib"
)
print("Kalibre edilmiş model kaydedildi.")

## Sonuç

| Yöntem | ECE |
|--------|-----|
| Ham Model | _(çalıştır)_ |
| Isotonic | _(çalıştır)_ |
| Sigmoid | _(çalıştır)_ |

**Kural:** ECE < 0.05 → kabul edilebilir kalibrasyon.

Bir sonraki notebook: `06_explainability.ipynb`